In [ ]:
import os

In [ ]:
# userhome = os.path.expanduser('~')
cwd = os.getcwd()
print(cwd)

In [ ]:
datafolder = cwd + '\\cas9_medfiles\\'
metafile = cwd + '\\CAS9_metafile.txt'



def extractlicks(self, substance):
    licks = medfilereader(self.medfile,
                              varsToExtract = sub2var(self, substance),
                                                remove_var_header = True)
    lickData = lickCalc(licks, burstThreshold=0.5, binsize=120)        

    return lickData

In [ ]:
def metafilereader(filename):
    
    f = open(filename, 'r')
    f.seek(0)
    header = f.readlines()[0]
    f.seek(0)
    filerows = f.readlines()[1:]
    
    tablerows = []
    
    for i in filerows:
        tablerows.append(i.split('\t'))
        
    header = header.split('\t')

    return tablerows, header

In [ ]:
def isnumeric(s):
    try:
        x = float(s)
        return x
    except ValueError:
        return float('nan')